## 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime
import numpy as np
import pickle
import pymysql
import warnings

from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings(action = 'ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 600)

## 데이터셋 구축 
- 보조지표 추가 X

In [37]:
def make_data_train(trading, days, label): 
    # traiding: 거래대금 임계값, days: 일 수, label: next_chang 임계 값
    
    # 코드 리스트 
    IF=open("../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    
    # trainX, trainY
    lst_code2date = []
    lst_trainX = []
    lst_trainY = []

    # database
    con = pymysql.connect(
            host = 'ahnbi2.suwon.ac.kr', 
            port = 3306, 
            user = 'stock_user', 
            passwd = 'bigdata', 
            db = 'stock_db',
            charset = 'utf8'
    )
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE (Date BETWEEN '2018-01-01' AND '2020-12-31') 
                    '''.format(code)
        stock = pd.read_sql(sql = sql_query, con = con)
        
        # 다음날 종가변화율
        stock['Next_Change'] = stock['Change'].shift(-1)
        stock.dropna(inplace=True) # nan 제거 
       
        lst_stock = stock.values.tolist()  
        
        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            
            
            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                lst_result = []
                
                Ddays_date = lst_stock[i-(days-1):i+1] # 10일간의 주가 정보
                                           
                for row2 in Ddays_date:
                    Open,High,Low,Close,Volume = row2[1:6]
                    Trading = Close * Volume
                    lst_result += [Open,High,Low,Close,Trading] 
                
                y = int(row[-1] >= label) # Next_Change
                 
                lst_code2date.append([code,date])
                lst_trainX.append(lst_result)
                lst_trainY.append(y)
    
    return lst_code2date, np.array(lst_trainX), np.array(lst_trainY)

In [62]:
def make_data_test(trading, days, label): 
    # 코드 리스트 
   
    IF=open("../data/code_list.txt", 'r')
    lst_code = IF.readlines()

    lst_code2date = []
    lst_testX = []
    lst_testY = []
    
    # database
    con = pymysql.connect(
            host = 'ahnbi2.suwon.ac.kr', 
            port = 3306, 
            user = 'stock_user', 
            passwd = 'bigdata', 
            db = 'stock_db',
            charset = 'utf8'
    )

    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE (Date BETWEEN '2021-01-01' AND '2021-12-31')
                    '''.format(code)
        stock = pd.read_sql(sql = sql_query, con = con)
        stock['Next_Change'] = stock['Change'].shift(-1)
        stock.dropna(inplace=True) # nan 제거         
        lst_stock = stock.values.tolist()  


        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            # 거래대금
            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                lst_result = []
                
                Ddays_date = lst_stock[i-(days-1):i+1]
                                           
                for row2 in Ddays_date:
                    Open,High,Low,Close,Volume = row2[1:6]
                    Trading = Close * Volume
                    lst_result += [Open,High,Low,Close,Trading]
                y = int(row[-1] >= label)
                
                lst_code2date.append([code,date])
                lst_testX.append(lst_result)
                lst_testY.append(y)
    
    return lst_code2date, np.array(lst_testX), np.array(lst_testY)

## 파일 저장 함수

In [39]:
def dump_file(TYPE, trading, days,target,code,data,y):
    with open('../data/{}_{}_{}_{}.pickle'.format(TYPE,trading, days,target), 'wb') as f:
        pickle.dump(code, f)
        pickle.dump(data, f)
        pickle.dump(y, f)

## 거래대금 1조, 10일, 다음날 상승률 0.02

In [40]:
train_code2date, lst_trainX, lst_trainY =  make_data_train(trading = 10000, days = 10,label =0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:57<00:00, 27.29it/s]


In [41]:
# 파일 저장
dump_file('train',10000,10,0.02,train_code2date,lst_trainX, lst_trainY)

In [42]:
test_code2date, lst_testX, lst_testY =  make_data_test(trading = 10000, days = 10,label =0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:29<00:00, 53.35it/s]


In [43]:
# 파일 저장
dump_file('test',10000,10,0.02,test_code2date,lst_testX, lst_testY)

In [44]:
len(lst_trainX), len(lst_testX)

(349, 275)

### 데이터프레임(참고용)

In [45]:
lst_col_info = []

for i in range(9,-1,-1):
    for val in ['Open','High','Low','Close','Volume']:
        lst_col_info.append(f'D-{i}_{val}')

In [46]:
lst_col_info[:5]

['D-9_Open', 'D-9_High', 'D-9_Low', 'D-9_Close', 'D-9_Volume']

In [47]:
df_trainX = pd.DataFrame(lst_trainX)
df_trainX.columns = lst_col_info
df_trainY = pd.DataFrame(lst_trainY)
df_trainX.head()

,D-9_Open,D-9_High,D-9_Low,D-9_Close,D-9_Volume,D-8_Open,D-8_High,D-8_Low,D-8_Close,D-8_Volume,D-7_Open,D-7_High,D-7_Low,D-7_Close,D-7_Volume,D-6_Open,D-6_High,D-6_Low,D-6_Close,D-6_Volume,D-5_Open,D-5_High,D-5_Low,D-5_Close,D-5_Volume,D-4_Open,D-4_High,D-4_Low,D-4_Close,D-4_Volume,D-3_Open,D-3_High,D-3_Low,D-3_Close,D-3_Volume,D-2_Open,D-2_High,D-2_Low,D-2_Close,D-2_Volume,D-1_Open,D-1_High,D-1_Low,D-1_Close,D-1_Volume,D-0_Open,D-0_High,D-0_Low,D-0_Close,D-0_Volume
0,13700,16100,13400,15400,897154258000,14700,15500,14000,14350,277027065700,13050,13300,11650,11650,231873876050,12200,13150,11600,12200,222393934200,12200,13750,12100,12350,256196958550,12500,15450,12450,14050,886535779600,14350,15300,13900,14350,333578205800,14450,14800,13550,13550,148350291550,13600,15400,13600,14050,451780054200,14600,18250,14550,18250,2343685750250
1,14700,15500,14000,14350,277027065700,13050,13300,11650,11650,231873876050,12200,13150,11600,12200,222393934200,12200,13750,12100,12350,256196958550,12500,15450,12450,14050,886535779600,14350,15300,13900,14350,333578205800,14450,14800,13550,13550,148350291550,13600,15400,13600,14050,451780054200,14600,18250,14550,18250,2343685750250,18500,21350,17500,18000,1963299564000
2,36379,40437,35662,38146,144505974164,38622,40102,38431,39864,62342791368,40437,40819,38670,40342,51753783592,40675,51560,40532,50320,271549816880,50988,51751,47646,50224,131053601568,51560,53852,47837,52134,167995611054,50606,52706,49746,51561,78144871941,52038,61682,51465,59391,394185728439,41630,47311,41630,41631,365547656460,38002,46595,33371,41726,1406528840666
3,58600,62900,34850,34850,808041161000,32900,33350,24800,29150,946589524100,28350,32450,27300,30200,686763371800,30550,30600,27100,27650,253417088750,27000,31000,25550,28250,547098427000,28350,28700,26300,26750,122904640500,26600,27300,25600,26150,96068510200,25750,30350,25050,26800,526937097600,27200,29900,26350,29700,282149346600,30900,35500,30250,34350,1016523775050
4,6650,7460,4460,4460,814557121960,4430,4835,3800,4290,760794181290,4370,4480,3935,4040,275440441080,4090,4210,3865,4040,135435994440,4265,4350,3935,4065,157481555250,4130,4135,3970,4030,69359479670,4010,4080,3850,3955,71513669290,3860,4025,3680,3915,100265976630,3800,4160,3730,4160,184787828160,4080,5190,3980,5190,1368703369410


In [48]:
df_code2date = pd.DataFrame(train_code2date, columns = ['code','date'])
df_trainY = pd.DataFrame(lst_trainY, columns = ['target'])
df = pd.concat([df_code2date, df_trainX, df_trainY], axis = 1)

In [49]:
df.head(10)

,code,date,D-9_Open,D-9_High,D-9_Low,D-9_Close,D-9_Volume,D-8_Open,D-8_High,D-8_Low,D-8_Close,D-8_Volume,D-7_Open,D-7_High,D-7_Low,D-7_Close,D-7_Volume,D-6_Open,D-6_High,D-6_Low,D-6_Close,D-6_Volume,D-5_Open,D-5_High,D-5_Low,D-5_Close,D-5_Volume,D-4_Open,D-4_High,D-4_Low,D-4_Close,D-4_Volume,D-3_Open,D-3_High,D-3_Low,D-3_Close,D-3_Volume,D-2_Open,D-2_High,D-2_Low,D-2_Close,D-2_Volume,D-1_Open,D-1_High,D-1_Low,D-1_Close,D-1_Volume,D-0_Open,D-0_High,D-0_Low,D-0_Close,D-0_Volume,target
0,050120,20180126,13700,16100,13400,15400,897154258000,14700,15500,14000,14350,277027065700,13050,13300,11650,11650,231873876050,12200,13150,11600,12200,222393934200,12200,13750,12100,12350,256196958550,12500,15450,12450,14050,886535779600,14350,15300,13900,14350,333578205800,14450,14800,13550,13550,148350291550,13600,15400,13600,14050,451780054200,14600,18250,14550,18250,2343685750250,0
1,050120,20180129,14700,15500,14000,14350,277027065700,13050,13300,11650,11650,231873876050,12200,13150,11600,12200,222393934200,12200,13750,12100,12350,256196958550,12500,15450,12450,14050,886535779600,14350,15300,13900,14350,333578205800,14450,14800,13550,13550,148350291550,13600,15400,13600,14050,451780054200,14600,18250,14550,18250,2343685750250,18500,21350,17500,18000,1963299564000,0
2,007390,20180320,36379,40437,35662,38146,144505974164,38622,40102,38431,39864,62342791368,40437,40819,38670,40342,51753783592,40675,51560,40532,50320,271549816880,50988,51751,47646,50224,131053601568,51560,53852,47837,52134,167995611054,50606,52706,49746,51561,78144871941,52038,61682,51465,59391,394185728439,41630,47311,41630,41631,365547656460,38002,46595,33371,41726,1406528840666,0
3,950130,20200903,58600,62900,34850,34850,808041161000,32900,33350,24800,29150,946589524100,28350,32450,27300,30200,686763371800,30550,30600,27100,27650,253417088750,27000,31000,25550,28250,547098427000,28350,28700,26300,26750,122904640500,26600,27300,25600,26150,96068510200,25750,30350,25050,26800,526937097600,27200,29900,26350,29700,282149346600,30900,35500,30250,34350,1016523775050,0
4,042040,20201014,6650,7460,4460,4460,814557121960,4430,4835,3800,4290,760794181290,4370,4480,3935,4040,275440441080,4090,4210,3865,4040,135435994440,4265,4350,3935,4065,157481555250,4130,4135,3970,4030,69359479670,4010,4080,3850,3955,71513669290,3860,4025,3680,3915,100265976630,3800,4160,3730,4160,184787828160,4080,5190,3980,5190,1368703369410,0
5,200230,20200924,5310,5380,5210,5350,7884246850,5370,5600,5370,5520,15643795920,5530,6700,5420,6050,423005563750,6340,7790,5810,7380,784999707120,7080,7920,6510,6740,714969676380,6880,7100,6530,6600,123909700200,6810,7700,6670,7130,498308220630,7260,7550,7010,7470,305307669780,8450,9710,8310,9710,804870299150,9500,12250,9010,9990,1668285344700,0
6,052260,20200519,25600,31400,25150,28750,694709796250,29150,29400,27200,27850,136044520700,27400,28700,27250,27800,86869328800,28150,31550,27850,29450,407392805400,30000,30800,28900,29500,91468585000,29600,30350,29050,30000,62817630000,29450,31100,29400,30650,106961327900,31200,31850,28900,29600,112011787200,29900,38450,29550,38450,588371358700,38600,48850,34800,40300,1774549969400,1
7,052420,20201013,2330,2395,2250,2255,3549257250,2280,2350,2230,2275,3448087825,2295,2390,2295,2370,2605087410,2400,2460,2380,2450,2544249050,2485,2485,2420,2450,1673472500,2450,2620,2435,2490,9716343540,2475,2580,2440,2525,6543103200,2585,2980,2520,2795,120987707230,2880,3630,2845,3630,261159388050,3880,4585,3595,4240,1042776941920,0
8,114120,20180221,15503,16601,15227,16331,72271778985,16649,16925,15775,15781,54789154383,15773,16417,15727,16377,34584915846,15912,16005,15682,15872,25605789696,15959,16143,15504,15506,28424668840,15550,16281,15457,15872,27818124288,15912,16326,15912,16056,26655625296,15272,16508,15134,16469,59547012707,16694,16694,16142,16194,28222951782,16234,21003,16188,19589,1241095199449,0
9,097780,20200827,7025,7375,7000,7350,17425255050,7350,7475,7125,7275,21438566550,7450,8425,6775,7450,277085510700,7550,7875,7325,7375,51185951500

In [50]:
df.shape

(349, 53)

## 보조지표 추가 데이터셋 구축

In [15]:
scaler = MinMaxScaler()

In [56]:
def make_data_train2(trading,  days, label): 
    # traiding: 거래대금 임계값, days: 일 수, label: next_chang 임계 값
    
    # 코드 리스트 
    IF=open("../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    
    # trainX, trainY
    lst_code2date = []
    lst_trainX = []
    lst_trainY = []
    
    # database
    db_dsml = pymysql.connect(
            host = 'ahnbi2.suwon.ac.kr', 
            port = 3306, 
            user = 'stock_user', 
            passwd = 'bigdata', 
            db = 'refined_stock', 
            charset = 'utf8'
        )
    
    
    cursor = db_dsml.cursor()
    
    
    col_scaling = ['Open', 'High', 'Low', 'Close', 'Trading', 'MA5', 'MA20', 'MA60', 'MA120', 
               'VMAP', 'BHB', 'BLB', 'KCH', 'KCL', 'KCM', 'DCH', 'DCL', 'DCM',
               'SMA', 'EMA', 'WMA', 'Ichimoku', 'Parabolic SAR', 'KAMA','MACD']   

    col_no_scaling = ['Change', 'Volume', 'MFI', 'ADI', 'OBV','CMF', 'FI', 'EOM, EMV', 
                      'VPT', 'NVI', 'ATR', 'UI', 'ADX', '-VI', '+VI', 'TRIX', 'MI', 
                      'CCI', 'DPO', 'KST', 'STC', 'RSI', 'SRSI', 'TSI', 'UO', 'SR',
                       'WR', 'AO', 'ROC', 'PPO', 'PVO']
       
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE Date BETWEEN '2018-01-01' AND '2020-12-31'
                    '''.format(code)

        stock = pd.read_sql(sql=sql_query, con=db_dsml)
        
        stock['BeforeClose'] = stock['Close'].shift(1) # 전날 종가 컬럼 추가
        stock.dropna(inplace=True) # nan 제거 
        lst_stock = stock.values.tolist()
    
        # scaling 필요 컬럼: 전날 종가로 나누어 스케일링
        stock['Trading'] = stock['Close'] * stock['Volume'] # 거래대금 컬럼 추가 
        scale_stock = stock[col_scaling].apply(lambda x: x/stock['BeforeClose'])
        lst_scale = scale_stock.values.tolist()

        # sclaling 필요x 컬럼 
        no_scale_stock = stock[col_no_scaling]
        lst_no_scale = no_scale_stock.values.tolist()        
        
        for i, row in enumerate(lst_stock): 
            date, close, volume,  = row[0].strftime('%Y%m%d'), row[4], row[5],
            # 거래대금, cci 범위 조건

            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               
                result=[]
                for row2, row3 in zip(lst_scale[i-(days-1):i+1], lst_no_scale[i-(days-1):i+1]):  # scaling, no scaling 
                    result += row2+row3 # scaling + no scaling                 
                y = int(row[7]>=label) # next change 사용   
                
                lst_code2date.append([code,date])
                lst_trainX.append(result)
                lst_trainY.append(y)
    
    return lst_code2date, scaler.fit_transform(np.array(lst_trainX)), np.array(lst_trainY)

In [63]:
def make_data_test2(trading,  days, label): 
    # traiding: 거래대금 임계값, days: 일 수, label: next_chang 임계 값
    
    # 코드 리스트 
    IF=open("../data/code_list.txt", 'r')
    lst_code = IF.readlines()
                    
    # testX, testY
    lst_code2date = []
    lst_testX = []
    lst_testY = []
    
    
    # database
    db_dsml = pymysql.connect(
            host = 'ahnbi2.suwon.ac.kr', 
            port = 3306, 
            user = 'stock_user', 
            passwd = 'bigdata', 
            db = 'refined_stock', 
            charset = 'utf8'
        )
    
    
    cursor = db_dsml.cursor()
    
    
    col_scaling = ['Open', 'High', 'Low', 'Close', 'Trading', 'MA5', 'MA20', 'MA60', 'MA120', 
               'VMAP', 'BHB', 'BLB', 'KCH', 'KCL', 'KCM', 'DCH', 'DCL', 'DCM',
               'SMA', 'EMA', 'WMA', 'Ichimoku', 'Parabolic SAR', 'KAMA','MACD']   

    col_no_scaling = ['Change', 'Volume', 'MFI', 'ADI', 'OBV','CMF', 'FI', 'EOM, EMV', 
                      'VPT', 'NVI', 'ATR', 'UI', 'ADX', '-VI', '+VI', 'TRIX', 'MI', 
                      'CCI', 'DPO', 'KST', 'STC', 'RSI', 'SRSI', 'TSI', 'UO', 'SR',
                       'WR', 'AO', 'ROC', 'PPO', 'PVO']
       
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE Date BETWEEN '2021-01-01' AND '2021-12-31'
                    '''.format(code)

        stock = pd.read_sql(sql=sql_query, con=db_dsml)
        
        stock['BeforeClose'] = stock['Close'].shift(1) # 전날 종가 컬럼 추가
        stock.dropna(inplace=True) # nan 제거 
        lst_stock = stock.values.tolist()
    
        # scaling 필요 컬럼: 전날 종가로 나누어 스케일링
        stock['Trading'] = stock['Close'] * stock['Volume'] # 거래대금 컬럼 추가 
        scale_stock = stock[col_scaling].apply(lambda x: x/stock['BeforeClose'])
        lst_scale = scale_stock.values.tolist()

        # sclaling 필요x 컬럼 
        no_scale_stock = stock[col_no_scaling]
        lst_no_scale = no_scale_stock.values.tolist()        
        
        for i, row in enumerate(lst_stock): 
            date, close, volume,  = row[0].strftime('%Y%m%d'), row[4], row[5],

            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               
                result=[]
                for row2, row3 in zip(lst_scale[i-(days-1):i+1], lst_no_scale[i-(days-1):i+1]):  # scaling, no scaling 
                    result += row2+row3 # scaling + no scaling                 
                y = int(row[7]>=label) # next change 사용   
                
                lst_code2date.append([code,date])
                lst_testX.append(result)
                lst_testY.append(y)
    
    return lst_code2date,scaler.fit_transform(np.array(lst_testX)), np.array(lst_testY)

## 거래대금 1조, 10일, 다음날 상승률 0.02
- 보조지표 추가
- 전날 종가로 나누어 스케일링

stock.columns 정보<br>
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change',
       'Next Change', 'MA5', 'MA20', 'MA60', 'MA120', 'MFI', 'ADI', 'OBV',
       'CMF', 'FI', 'EOM, EMV', 'VPT', 'NVI', 'VMAP', 'ATR', 'BHB', 'BLB',
       'KCH', 'KCL', 'KCM', 'DCH', 'DCL', 'DCM', 'UI', 'SMA', 'EMA', 'WMA',
       'MACD', 'ADX', '-VI', '+VI', 'TRIX', 'MI', 'CCI', 'DPO', 'KST',
       'Ichimoku', 'Parabolic SAR', 'STC', 'RSI', 'SRSI', 'TSI', 'UO', 'SR',
       'WR', 'AO', 'KAMA', 'ROC', 'PPO', 'PVO', 'BeforeClose'],

In [58]:
train_code2date, lst_trainX, lst_trainY =  make_data_train2(trading = 10000, days = 10,label =0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [03:16<00:00,  7.96it/s]


In [59]:
# 파일 저장
dump_file('indicator_train',10000,10,0.02,train_code2date,lst_trainX, lst_trainY)

In [60]:
test_code2date, lst_testX, lst_testY =  make_data_test2(trading = 10000, days = 10,label =0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:28<00:00, 17.61it/s]


In [61]:
# 파일 저장
dump_file('indicator_test',10000,10,0.02,test_code2date,lst_testX, lst_testY)